# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

import pyro
import math
import pickle
import time
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

import os

from torch.utils.data import TensorDataset, DataLoader
import itertools

In [2]:
import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.NN_models as NN_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [3]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('Data/X_test.csv', header=None, delimiter=',').values

Y_train_21 = pd.read_csv('Data/Y_train_std_21.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('Data/Y_test_std_21.csv', header=None, delimiter=',').values

Y_train = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',').values
Y_test = pd.read_csv('Data/Y_test_std.csv', header=None, delimiter=',').values


train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_21 = torch.tensor(Y_train_21, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_21, dtype=torch.float32)

# train_y = torch.tensor(Y_train, dtype=torch.float32)
# test_y = torch.tensor(Y_test, dtype=torch.float32)


# torch.set_default_dtype(torch.float32)

# Model

In [ ]:
NN_4 = Training.train_DNN_MSE(NN_models.NN_4,
                              train_x,
                              train_y_21,
                              num_iterations= 50000,
                              device= 'cuda',
                              show_progress = True,
                              weight_decay = 0,
                              val_x=test_x,
                              val_y=test_y_21,
                              early_stopping = True,
                              patience = 1000,
                              val_check_interval = 100)

In [ ]:
NN_5 = Training.train_DNN_MSE(NN_models.NN_5,
                              train_x,
                              train_y_21,
                              num_iterations= 50000,
                              device= 'cuda',
                              show_progress = True,
                              weight_decay = 0,
                              val_x=test_x,
                              val_y=test_y_21,
                              early_stopping = True,
                              patience = 1000,
                              val_check_interval = 100)

In [5]:
BNN_Deep, Posterior_BNN_Deep = Training.train_BNN_minibatch(NN_models.BNN_Deep,
    train_x,
    train_y_21,
    num_iterations=50000,
    batch_size=256,
    device='cuda',
    show_progress=True,
    val_x=test_x,
    val_y=test_y_21,
    early_stopping=True,
    patience=1000,
    val_check_interval=100)

SVI training:   5%|▌         | 2551/50000 [02:48<1:01:31, 12.85it/s]

KeyboardInterrupt: 

SVI training:   5%|▌         | 2551/50000 [03:00<1:01:31, 12.85it/s]

In [ ]:
BNN_WideDrop, Posterior_BNN_WideDrop = Training.train_BNN_minibatch(NN_models.BNN_WideDrop,
    train_x,
    train_y_21,
    num_iterations=50000,
    batch_size=256,
    device='cuda',
    show_progress=True,
    val_x=test_x,
    val_y=test_y_21,
    early_stopping=True,
    patience=1000,
    val_check_interval=100)

In [ ]:
BNN_ARD, Posterior_BNN_ARD = Training.train_BNN_minibatch(NN_models.BNN_ARD,
    train_x,
    train_y_21,
    num_iterations=50000,
    batch_size=256,
    device='cuda',
    show_progress=True,
    val_x=test_x,
    val_y=test_y_21,
    early_stopping=True,
    patience=1000,
    val_check_interval=100)

In [ ]:
import torch
import torch.nn.functional as F
from torch.distributions import Distribution

def evaluate_bnn_mse(model, test_x, test_y, device='cuda'):

    model.eval()
    test_x = test_x.to(device)
    test_y = test_y.to(device)

    with torch.no_grad():
        raw_out = model(test_x)
        if isinstance(raw_out, Distribution):
            preds = raw_out.mean
        else:
            preds = raw_out
    # MSE = mean( (preds - test_y)^2 )
    mse = F.mse_loss(preds, test_y)
    return mse.item()


mse_value = evaluate_bnn_mse(BNN_2, test_x, test_y_21, device='cuda')
print(f"Test MSE = {mse_value:.6f}")

# Paramater inference

In [ ]:
row_idx = 0
input_point = test_y_21[row_idx, :]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_21, k=100)


## Point

In [ ]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)
    ).cpu().detach().numpy()


bounds = bound.get_bounds(local_train_x)

estimated_params_tmp, _ = Estimation.multi_start_estimation(
    MVGP_models, MVGP_likelihoods, row_idx, test_y_21, bounds,
    Estimation.estimate_params_for_one_model_Adam, num_starts=4, num_iterations=1000, lr=0.01,
    patience=10, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
)

with open(output_file, 'a') as f:
    # f.write(f"{row_idx + 1},\"{list(preds_tmp)}\",\"{list(estimated_params_tmp.detach().numpy())}\"\n")
    f.write(f"{row_idx + 1},\"{list(preds_tmp)}\",\"{list(estimated_params_tmp)}\"\n")

mcmc_result_Uniform = Estimation.run_mcmc_Uniform(
    Prediction.preds_distribution, MVGP_models, MVGP_likelihoods, 
    row_idx, test_y_21, bounds, 
    num_sampling=1200, warmup_step=300, num_chains=1, device=Device
)
posterior_samples_Uniform = mcmc_result_Uniform.get_samples()

## MCMC